# Sistema de recomendação de filtragem colaborativa - ALS

O sistema funcionará da seguinte maneira:
<br><br>
Os usuários dão rating (fazem uma avaliação) para alguns filmes e o algoritmo gera uma lista de outros filmes que o usuário também daria um bom rating. O ALS é um método de fatoração de matriz que irá 'preencher' os ratings dos filmes que o usuário não avaliou, fará isso baseando-se nos ratings que vários usuários deram aos filmes.<br>

Como o sistema irá funcionar?<br>
Cada conjunto usuário e filme será transformado em um vetor de números, os quais serão ajustados para reprensentar o interesse do usuário por uma característica de um filme.<br>

Esse é o algoritmo que ficou famoso no prêmio **Netflix** !!! Massa né. 
<hr>
Os dados utilizados é o do Group lens e pode ser encontrado no link abaixo.<br>
https://grouplens.org/

## Análise e Preparação dos Dados:

In [1]:
from __future__ import print_function

import sys
import pandas as pd
if sys.version >= '3':
    long = int
    
from pyspark.sql import SparkSession

# Verifica a qualidade do modelo.
from pyspark.ml.evaluation import RegressionEvaluator
# Modelo de recomendação.
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder\
    .appName("AlS")\
    .getOrCreate()

In [3]:
# Faz a leitura do arquivo csv com o pandas.
df = pd.read_csv('ratings.csv')

In [4]:
# Imprime na tela o data frame.
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [5]:
# Converte o pandas dataframe no spark dataframe para ganhar mais desempenho.
ratings = spark.createDataFrame(df)

In [6]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



## Aplicar o algoritmo de Machine Learning ALS

In [7]:
# Quebra o data frame com 80% para treino e 20% para teste.
(training, test) = ratings.randomSplit([0.8,0.2])

Observações:<br>
<ol>
    <li>maxIter: Número máximo de iterações.</li>
    <li>regParam: Coeficiente de aprendizado.</li>
    <li>userCol: Coluna de usuário</li>
    <li>itemCol: Coluna de itens.</li>
    <li>ratingCol: Coluna de rating.</li>
    <li>coldStarStrategy: Se houver problema de recomendação para o usuário vamos dropar.</li>
</ol>

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

In [9]:
# Fazer a predição no conjunto de teste.
predictions = model.transform(test)
# Avaliar o conjunto de teste.
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-men-square error = "+ str(rmse))

Root-men-square error = 1.091061231039561


In [10]:
# Recomendação para o usuário. A linha abaixo gera dez recomendações para o usuário.
# Caso seja necessário um número maior de recomendação de usuário é só alterar o paramêtro 10.
userRecs = model.recommendForAllUsers(10)

In [11]:
# A saída terá o identificador do usuário, do filme e o rating que o usuário daria para o filme.
userRecs.show(10, False)

+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                          |
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|471   |[[4678, 7.349183], [2459, 7.097487], [2171, 6.9601874], [34338, 6.8947434], [2901, 6.840605], [175303, 6.638791], [1883, 6.5019073], [7096, 6.4062405], [3703, 6.380869], [3969, 6.3573327]]             |
|463   |[[112623, 6.933939], [1218, 6.63095], [102903, 6.3386674], [53123, 6.0316443], [81847, 6.0101047], [52885, 6.0020437], [135143, 5.9997215], [142488,

In [12]:
# Pegar o usuário para um produto específico. Utilizando a transposta.
# Caso seja necessário um número maior de recomendação de filme é só alterar o paramêtro 10.
movieRecs = model.recommendForAllItems(10)

In [13]:
movieRecs.show(10, False)

+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movieId|recommendations                                                                                                                                                               |
+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1580   |[[569, 6.6508803], [536, 6.60852], [340, 6.4993625], [584, 5.976654], [192, 5.974021], [38, 5.953698], [126, 5.8064537], [507, 5.791917], [337, 5.7715535], [35, 5.6274734]]  |
|4900   |[[81, 5.9313197], [127, 5.4344397], [77, 5.3131638], [375, 5.18896], [99, 4.6750474], [112, 4.6719], [531, 4.563199], [277, 4.5542254], [276, 4.485346], [353, 4.4572144]]    |
|5300   |[[461, 7.4617023], [360, 6.1435757], [392, 6.053887], [531, 5.9716

In [14]:
# Selecionar os usuários existentes.
users = ratings.select(als.getUserCol()).distinct()

In [15]:
users.show()

+------+
|userId|
+------+
|    26|
|    29|
|   474|
|    65|
|   191|
|   418|
|   541|
|   558|
|   222|
|   270|
|   293|
|   243|
|   278|
|   367|
|   442|
|    19|
|    54|
|   296|
|   277|
|   287|
+------+
only showing top 20 rows



In [16]:
# Recomendações mas sem o rating para ficar mais limpo, até porque não é necessário mostrar o rating.
userRecsOnlyItemId = userRecs.select(userRecs['userId'], userRecs['recommendations']['movieId'])

In [17]:
userRecsOnlyItemId.show(10,False)

+------+--------------------------------------------------------------------------+
|userId|recommendations.movieId                                                   |
+------+--------------------------------------------------------------------------+
|471   |[4678, 2459, 2171, 34338, 2901, 175303, 1883, 7096, 3703, 3969]           |
|463   |[112623, 1218, 102903, 53123, 81847, 52885, 135143, 142488, 166461, 64716]|
|496   |[4649, 5048, 89118, 86911, 69278, 319, 556, 5909, 1186, 6666]             |
|148   |[7381, 6380, 3265, 86911, 130490, 55052, 1218, 166461, 1241, 40962]       |
|540   |[53123, 945, 86911, 905, 34338, 6666, 3265, 1237, 2131, 161582]           |
|392   |[4649, 70994, 5048, 86911, 383, 3513, 5909, 3096, 2130, 1952]             |
|243   |[5650, 3358, 5666, 1866, 185029, 174, 2318, 2393, 183897, 44788]          |
|31    |[232, 2133, 3513, 2431, 3506, 1957, 3524, 4121, 179819, 905]              |
|516   |[140110, 905, 7587, 3265, 56145, 1212, 2459, 839, 4437, 6857]       

## Conexão com o Banco de Dados

Será utilizado o MongoDB é um software de banco de dados orientado a documentos livre, de código aberto e multiplataforma, escrito na linguagem C++.[5] Classificado como um programa de banco de dados NoSQL, o MongoDB usa documentos semelhantes a JSON com esquemas. É desenvolvido pela MongoDB Inc. e publicado sob uma combinação da GNU Affero General Public License e Licença Apache.

Suas características permitem com que as aplicações modelem informações de modo muito mais natural, pois os dados podem ser aninhados em hierarquias complexas e continuar a ser indexáveis e fáceis de buscar.

Caso não possua o Mongodb instalado:<br>
https://docs.mongodb.com/manual/administration/install-community/

#### Colocando os dados no Mongodb

É necessário ter a biblioteca pymongo para colocar os dados no Mongodb. Logo caso não tenha abra o terminal e execute **pip install pymongo**.

In [18]:
# Criando uma variavel para armazenar as recomendações.
recomendationToSave = model.recommendForAllUsers(10)
# Adaptando a variável da recomendations e movieId para uma única variável.
recomendationToSave = recomendationToSave.select(recomendationToSave['userId'], recomendationToSave['recommendations']['movieId'].alias('movieId'))

In [19]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost',27017)
db = client.pchp

In [ ]:
# Coleta a recomendação para a variável colecao.
colecao = recomendationToSave.collect()

# Insere uma a uma no Mongodb
for row in colecao:
    db.suggestions.insert_one(row.asDict())

In [ ]:
recomendationToSave.show()

Requisição no banco para mostrar que deu tudo certo.
<img src='suggestions_mongodb.png' /></a>

# Próximo passo fazer o Front para conectar com o Banco de Dados !!!